In [1]:
import sys

sys.path.append("..")

import numpy as np

%matplotlib inline 

import gc
import json

import torch

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
from tqdm.auto import tqdm

from src.tools import get_loader_stats, load_dataset

LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

/home/zyz/DiffOT/.venv/lib/python3.12/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gc.collect()
torch.cuda.empty_cache()

## Main Config


In [3]:
DEVICE_ID = 0

DATASET_LIST = [
    ("MNIST-colored_3", "~/data/MNIST", 32, 1),
]
# DATASET_LIST = [
#     ("CelebA_high", "~/data/img_align_celeba", 64, 1),
# ]

assert torch.cuda.is_available()
torch.cuda.set_device(f"cuda:{DEVICE_ID}")

In [4]:
for DATASET, DATASET_PATH, IMG_SIZE, N_EPOCHS in tqdm(DATASET_LIST):
    print("Processing {}".format(DATASET))
    sampler, test_sampler = load_dataset(
        DATASET, DATASET_PATH, img_size=IMG_SIZE, batch_size=256
    )
    print("Dataset {} loaded".format(DATASET))

    mu, sigma = get_loader_stats(
        test_sampler.loader, n_epochs=N_EPOCHS, verbose=True, batch_size=256
    )
    print("Trace of sigma: {}".format(np.trace(sigma)))
    stats = {"mu": mu.tolist(), "sigma": sigma.tolist()}
    print("Stats computed")

    filename = "{}_{}_test.json".format(DATASET, IMG_SIZE)
    with open(filename, "w") as fp:
        json.dump(stats, fp)
    print("States saved to {}".format(filename))

  0%|                                                                                                                                                                                                              | 0/1 [00:00<?, ?it/s]

Processing MNIST-colored_3
Dataset MNIST-colored_3 loaded


/home/zyz/DiffOT/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zyz/DiffOT/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)

0it [00:00, ?it/s]
1it [00:00,  3.10it/s]
2it [00:00,  4.85it/s]
3it [00:00,  5.98it/s]
4it [00:00,  5.77it/s]


Trace of sigma: 67.06710443320588
Stats computed


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.25s/it]

States saved to MNIST-colored_3_32_test.json
